In [3]:
%pip install pyod scikit-learn

import pandas as pd
from pyod.models.knn import KNN
from sklearn.preprocessing import StandardScaler



In [4]:
covidtotals = pd.read_csv('data/covidtotals.csv')
covidtotals.set_index('iso_code', inplace=True)

In [5]:
standardizer = StandardScaler()
analysisvars = ['location', 'total_cases_pm', 'total_deaths_pm', 'pop_density', 'median_age', 'gdp_per_capita']
covidanalysis = covidtotals.loc[:, analysisvars].dropna()
covidanalysisstand = standardizer.fit_transform(covidanalysis.iloc[:, 1:])

In [6]:
clf_name = 'KNN'
clf = KNN(contamination=0.1)
clf.fit(covidanalysisstand)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [8]:
y_pred = clf.labels_
y_scores = clf.decision_scores_


In [9]:
pred = pd.DataFrame(zip(y_pred, y_scores), columns=['outliers', 'scores'], index=covidanalysis.index)

pred.sample(10, random_state=2)


,outliers,scores
iso_code,,
BHR,1,2.694270
BRA,0,0.752140
ZWE,0,0.210114
BGR,1,1.619251
CHN,0,0.935879
BGD,1,1.520910
GRD,0,0.681296
UZB,0,0.369320
MMR,0,0.370487


In [10]:
pred.outliers.value_counts()

outliers
0    162
1     18
Name: count, dtype: int64

In [11]:
pred.groupby(['outliers'])[['scores']].agg(['min', 'median', 'max'])

scores                     
               min    median        max
outliers                               
0         0.078210  0.600947   1.397616
1         1.421653  1.653276  11.940163

In [12]:
covidanalysis.join(pred).loc[pred.outliers == 1, ['location', 'total_cases_pm', 'total_deaths_pm','scores']].sort_values(by='scores', ascending=False).head(10)

,location,total_cases_pm,total_deaths_pm,scores
iso_code,,,,
SGP,Singapore,531183.841,346.637,11.940163
QAT,Qatar,190908.716,256.017,3.037338
BHR,Bahrain,473167.024,1043.310,2.694270
LUX,Luxembourg,603439.463,1544.161,2.485225
PER,Peru,133238.998,6507.656,2.265343
BRN,Brunei,763475.441,396.435,2.259517
MDV,Maldives,356423.660,603.286,1.984898
MLT,Malta,227422.824,1687.628,1.961672
ARE,United Arab Emirates,113019.214,248.805,1.687302
